In [1]:
import cobrakbase

cobrakbase 0.2.7


In [2]:
kbase = cobrakbase.KBaseAPI()

In [3]:
template = kbase.get_from_ws('CoreModelTemplateV3', 'NewKBaseModelTemplates')

In [4]:
type(template)

cobrakbase.core.kbasefba.newmodeltemplate.NewModelTemplate

In [19]:
rxn = template.reactions.get_by_id('rxn00265_c')

In [111]:
model['modelcompounds'][0]

{'aliases': [],
 'charge': -3,
 'compound_ref': '~/template/compounds/id/cpd00137',
 'dblinks': {},
 'formula': 'C6H5O7',
 'id': 'cpd00137_c0',
 'inchikey': 'KRKNYBCHXYNGOX-UHFFFAOYSA-K',
 'modelcompartment_ref': '~/modelcompartments/id/c0',
 'name': 'Citrate_c0',
 'numerical_attributes': {},
 'smiles': 'O=C([O-])CC(O)(CC(=O)[O-])C(=O)[O-]',
 'string_attributes': {}}

In [101]:
template.compcompounds[0]

{'charge': -4,
 'id': 'cpd00290_c',
 'maxuptake': 0,
 'templatecompartment_ref': '~/compartments/id/c',
 'templatecompound_ref': '~/compounds/id/cpd00290'}

In [116]:
def NewModelTemplateCompCompoundToMetabolite(template_comp_compound, index='0'):
    compound = template_reaction.template.compounds.get_by_id(template_comp_compound['templatecompound_ref'].split('/')[-1])
    compartment = template_comp_compound.templatecompartment_ref.split('/')[-1]
    cpd = ModelCompound(
        {
            'id': template_comp_compound.id + str(index), 
            'modelcompartment_ref': template_comp_compound.templatecompartment_ref + str(index), 
            'formula': compound.formula, 
            'name': "{}_{}{}".format(compound.name, compartment, index), 
            'charge': template_comp_compound.charge,
            'string_attributes': {},
            'numerical_attributes': {},
            'smiles': {},
            'dblinks': {},
            'aliases': {}
        })
    return cpd
NewModelTemplateCompCompoundToMetabolite(template.compcompounds.get_by_id('cpd00001_c'), '0')

Metabolite identifier,cpd00001_c0
Name,H2O_c0
Memory address,0x07fc150372ed0
Formula,H2O
Compartment,c0
In 0 reaction(s),


In [103]:
from cobrakbase.core.kbasefba.fbamodel_reaction import ModelReaction
from cobrakbase.core.kbasefba.fbamodel_metabolite import ModelCompound

def NewModelTemplateReactionToReaction(template_reaction, index='0'):
    reaction_compartment = 'c'
    model_reaction_data = {
        'id': "{}{}".format(template_reaction.id, index),
        'aliases': [],
        'coverage': 3,
        'dblinks': {},
        'direction': template_reaction.direction,
        'edits': {},
        'gapfill_data': {},
        'gene_count': 3,
        'maxforflux': 1000000,
        'maxrevflux': 1000000,
        'modelReactionProteins': [],
        'modelReactionReagents': [
            {'coefficient': -1, 'modelcompound_ref': '~/modelcompounds/id/cpd00137_c0'},
            {'coefficient': 1, 'modelcompound_ref': '~/modelcompounds/id/cpd00029_c0'},
            {'coefficient': 1, 'modelcompound_ref': '~/modelcompounds/id/cpd00032_c0'}
        ],
        'modelcompartment_ref': '~/modelcompartments/id/c0',
        'name': "{}_{}{}".format(template_reaction.name, reaction_compartment, index),
        'numerical_attributes': {},
        'probability': 1,
        'protons': 0,
        'reaction_ref': '~/template/reactions/id/rxn00265_c',
        'string_attributes': {}
    }
    metabolites = {}
    for o in template_reaction.templateReactionReagents:
        comp_compound = template_reaction.template.compcompounds.get_by_id(o['templatecompcompound_ref'].split('/')[-1])
        compound = template_reaction.template.compounds.get_by_id(comp_compound['templatecompound_ref'].split('/')[-1])
        cpd = ModelCompound(
            {
                'id': comp_compound.id + str(index), 
                'modelcompartment_ref': comp_compound.templatecompartment_ref + str(index), 
                'formula': compound.formula, 
                'name': compound.name, 
                'charge': comp_compound.charge
            })
        metabolites[cpd] = o['coefficient']
    reaction = ModelReaction(model_reaction_data)
    reaction.add_metabolites(metabolites)
    return reaction
template_reaction = template.reactions.get_by_id('rxn00265_c')
model_reaction = NewModelTemplateReactionToReaction(template_reaction, 9)
print(model_reaction)

rxn00265_c9: cpd00137_c9 --> cpd00029_c9 + cpd00032_c9


In [29]:
model = kbase.get_object('ecoli.core', 'filipeliu:narrative_1582914694010')

In [88]:
model_reaction

Reaction identifier,rxn00265_c9
Name,citrate oxaloacetate-lyase (forming acetate from the pro-S carboxymethyl group of citrate)_c9
Memory address,0x07fc17063d290
Stoichiometry,cpd00137_c --> cpd00029_c + cpd00032_c Citrate --> Acetate + Oxaloacetate
GPR,
Lower bound,0
Upper bound,1000


In [22]:
rxn1 = model.reactions.get_by_id('rxn00265_c0')

In [184]:
from cobrakbase.core.kbasefba.fbamodel_builder import FBAModelBuilder

def build_model_from_some_map(rxn_ids, template, index='0'):
    model_base = {
        'id': 'ecoli.core',
        'name': 'ecoli.core',
        'modelcompounds': [],
        'modelreactions': [],
        '__VERSION__': 1,
        'attributes': {
            'auxotrophy': {},
            'fbas': {},
            'gene_count': 0,
            'pathways': {}
        },
        'biomasses': [],
        'contig_coverages': {},
        'delete_biomasses': {},
        'deleted_reactions': {},
        'gapfilledcandidates': [],
        'gapfillings': [],
        'gapgens': [],
        'genome_ref': '56739/4/1',
        'loops': [],
        'model_edits': [],
        'modelcompartments': [{'compartmentIndex': 0,
       'compartment_ref': '~/template/compartments/id/c',
       'id': 'c0',
       'label': 'Cytosol_0',
       'pH': 7,
       'potential': 0},
      {'compartmentIndex': 0,
       'compartment_ref': '~/template/compartments/id/e',
       'id': 'e0',
       'label': 'Extracellular_0',
       'pH': 7,
       'potential': 0}],
        'other_genome_refs': [],
        'quantopts': [],
        'source': 'KBase',
        'source_id': 'ecoli.core',
        'template_ref': '56739/219/6',
        'template_refs': ['56739/219/6'],
        'type': 'GenomeScale'
    }
    reactions = []
    for rxn_id in rxn_ids:
        template_reaction = template.reactions.get_by_id(rxn_id)
        model_reaction = NewModelTemplateReactionToReaction(template_reaction, index)
        reactions.append(model_reaction)
    b = FBAModelBuilder(model_base)
    cobra_model = b.build()
    cobra_model.add_reactions(reactions)
    return cobra_model
cobra_model = build_model_from_some_map(['rxn14425_c', 'rxn14426_c'], template)

In [185]:
from cobra.core import Reaction, Model
class exchanger():
    
    def __init__(self, cobra_model):
        self.model = Model(cobra_model)
        exchanges = []
        for m in cobra_model.metabolites:
            r = Reaction('EX_' + m.id, 'Exchange for ' + m.name, 'debug', 0, 1000)
            r.add_metabolites({m:-1})
            exchanges.append(r)
        self.model.add_reactions(exchanges)

ex = exchanger(cobra_model)

In [187]:
cobra_model.objective = 'rxn14425_c0'

In [189]:
ex.model.medium = {
    'EX_cpd00007_c0': 0.5,
    'EX_cpd00067_e0': 10,
    'EX_cpd00067_c0': 10,
    'EX_cpd00001_c0': 10,
    'EX_cpd15561_c0': 10,
    'EX_cpd15560_c0': 10,
}
ex.model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
cpd00007_c0,EX_cpd00007_c0,0.5,0,0.00%
cpd00067_c0,EX_cpd00067_c0,4,0,0.00%
cpd15561_c0,EX_cpd15561_c0,1,49,100.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
cpd00001_c0,EX_cpd00001_c0,-1,0,0.00%
cpd00067_e0,EX_cpd00067_e0,-4,0,0.00%
cpd15560_c0,EX_cpd15560_c0,-1,49,100.00%
